In [1]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk
from itk import TubeTK as ttk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [6]:
def extract_columns(pathname, output_pathname, min_size=16):
    data_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.mha"))
    data_overlay_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.columns-roi-overlay.mha"))
        
    num_overlays = len(data_overlay_paths)
    for i in range(num_overlays):
        img = itk.imread(data_paths[i], itk.F)
        ovrimg = itk.imread(data_overlay_paths[i], itk.UC)
        print("Processing", data_overlay_paths[i])
        
        nos = 0
        rib = 0
        sli = 0
        c = 0
        while c<ovrimg.shape[2]:
            nos = np.count_nonzero(ovrimg[:,:,c]==1)
            rib = np.count_nonzero(ovrimg[:,:,c]==2)
            sli = np.count_nonzero(ovrimg[:,:,c]==3)
            start_c = 0
            end_c = 0
            val_c = 0
            name_c = ''
            if nos>0 and rib==0 and sli==0:
                val_c = 1
                name_c = "N"
            elif nos==0 and rib>0 and sli==0:
                val_c = 2
                name_c = "R"
            elif nos==0 and rib==0 and sli>0:
                val_c = 3
                name_c = "S"
            else:
                c += 1
                continue
            start_c = c
            end_c = c
            c += 1
            count = 0
            if c<ovrimg.shape[2]:
                count = np.count_nonzero(ovrimg[:,:,c]==val_c)
                while count>0 and c<ovrimg.shape[2]-1:
                    c += 1
                    count = np.count_nonzero(ovrimg[:,:,c]==val_c)
            end_c = c
            if end_c-start_c>=min_size:
                (filepath,filename) = os.path.split(data_overlay_paths[i])
                
                new_name = output_pathname+"/"+filename.split('.')[0]+'_Class'+name_c+'_'+str(start_c)+'-'+str(end_c)+'.mha'
                outimg = img[:,:,start_c:end_c]
                itkimg = itk.GetImageFromArray(outimg.astype(np.short))
                itk.imwrite(itkimg, new_name, compression=True)
                
                new_name = output_pathname+"/"+filename.split('.')[0]+'_Class'+name_c+'_'+str(start_c)+'-'+str(end_c)+'.roi-overlay.mha'
                outimg = ovrimg[:,:,start_c:end_c]
                outimg = np.where(outimg==3,1,0)
                itkimg = itk.GetImageFromArray(outimg.astype(np.ubyte))
                itk.imwrite(itkimg, new_name, compression=True)

In [10]:
output_pathname = "../../Data/VFoldData/ColumnData"

#pathname = "../../Data/VFoldData/BAMC-PTXNoSliding"
pathname = "../../Data/VFoldData/BAMC-PTXSliding"

extract_columns(pathname, output_pathname, 16)

Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_73815992352100_clean.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_74132233134844_clean.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10705997566592_CLEAN.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10891015221417_clean.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/030s_iimage_1180496934444_clean.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/030s_iimage_677741729740_clean.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/034s_iimage_3368391807672_clean.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/034s_iimage_3401832241774_clean.columns-roi-overlay.mha
Processing ..